In [9]:
import requests
import pandas as pd
from io import StringIO
import os, sys

In [3]:
# URL of the CSV file in Google Drive (make sure the sharing settings are set to "Anyone with the link")
url = 'https://drive.google.com/uc?id=1G4MUbdUFeACgjGcg6dcwykBZn21zg1jO'

Response = requests.get(url)

# Download the CSV file into memory
csv_content = Response.text

# Read the CSV data from the downloaded content
df = pd.read_csv(StringIO(csv_content))

In [ ]:
df.info()

In [4]:
df.drop('EVENT_NO_STOP', axis=1, inplace=True)

In [ ]:
df.info()

In [5]:
df.drop(['GPS_SATELLITES', 'GPS_HDOP'], axis=1, inplace=True)

In [ ]:
columns = ['EVENT_NO_TRIP', 'OPD_DATE', 'VEHICLE_ID', 'METERS', 'ACT_TIME', 'GPS_LONGITUDE', 'GPS_LATITUDE']
df1 = pd.read_csv(StringIO(csv_content), usecols=columns)
df1.info()


In [14]:
def timestamp(df1):
  # Create an empty list to store Timestamp objects
  timestamps = []

  # Iterate over DataFrame rows using iterrows()
  for index, row in df1.iterrows():
      df_total = int(row['ACT_TIME'])
      df_hour = df_total // (60*60)
      df_min = (df_total - (df_hour*3600)) // 60
      df_sec = df_total - (df_hour*3600) - (df_min*60)
      df_month = pd.to_datetime(row['OPD_DATE'], format="%d%b%Y:%H:%M:%S").month
      df_day = pd.to_datetime(row['OPD_DATE'], format="%d%b%Y:%H:%M:%S").day

      # Create a Timestamp object and append it to the list
      timestamp = pd.Timestamp(year=2019, month=df_month, day=df_day, hour=df_hour, minute=df_min, second=df_sec)
      timestamps.append(timestamp)

  # Assign the list of Timestamps to the 'TIMESTAMP' column of the DataFrame
  df1['TIMESTAMP'] = timestamps
  return df1


In [8]:
df1.drop(['ACT_TIME', 'OPD_DATE'], axis=1, inplace=True)

In [ ]:
print(df1)

In [27]:
def speed(df1):
  dMETERS = []
  dTIMESTAMPS = []
  SPEED = []

  for index, row in df1.iterrows():

      if index > 0 and df1['EVENT_NO_TRIP'].diff != 0:
          dMETER = df1.at[index, 'METERS'] - df1.at[index - 1, 'METERS']
      else:
          dMETER = 0

      if index > 0 and df1['EVENT_NO_TRIP'].diff != 0:
          dTIMESTAMP = (df1.at[index, 'TIMESTAMP'] - df1.at[index - 1, 'TIMESTAMP']).total_seconds()
      else:
          dTIMESTAMP = 0

      if dTIMESTAMP != 0 and df1['EVENT_NO_TRIP'].diff != 0:
          dSPEED = dMETER / dTIMESTAMP
      else:
          dSPEED = 0

      dMETERS.append(dMETER)
      dTIMESTAMPS.append(dTIMESTAMP)
      SPEED.append(dSPEED)


  df1['dMETERS'] = dMETERS
  df1['dTIMESTAMP'] = dTIMESTAMPS
  df1['SPEED'] = SPEED
  return df1

In [ ]:
print(df1)

In [21]:
file_path = "/content/bc_trip259172515_230215.csv"
if os.path.isfile(file_path) == False:
  !gdown 133NksS6c4yW2rs-U8OwrncjoKzJtC4JL

# Concatenate the list of chunks into a single DataFrame
Ldf = pd.read_csv(file_path, low_memory=False)

#Part C
Ldf.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis=1, inplace=True)

In [ ]:
timestamp(Ldf)
Ldf.info()

In [32]:
speed(Ldf)
Ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   EVENT_NO_TRIP  161 non-null    int64         
 1   OPD_DATE       161 non-null    object        
 2   VEHICLE_ID     161 non-null    int64         
 3   METERS         161 non-null    int64         
 4   ACT_TIME       161 non-null    int64         
 5   GPS_LONGITUDE  161 non-null    float64       
 6   GPS_LATITUDE   161 non-null    float64       
 7   TIMESTAMP      161 non-null    datetime64[ns]
 8   dMETERS        161 non-null    int64         
 9   dTIMESTAMP     161 non-null    float64       
 10  SPEED          161 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(5), object(1)
memory usage: 14.0+ KB


In [ ]:
Ldf.drop(['ACT_TIME', 'OPD_DATE'], axis=1, inplace=True )